This script will generate youtube short and long contents based on the ChatGPT output or video link you provided.

media/*  directories must contain image files that will be used for this purposes.

*All the videos in this youtube channel created by this script : https://www.youtube.com/@Immortal_Stoic*


In [ ]:
# ----------------------------  FILL THIS PARAMETERS !!!!  ----------------------
IMAGE_VIDEO = "image"                                                   # Create a video from images or videos
CONTENT = "normal"                                                      # shorts / normal
YOUTUBE_LINK = "https://www.youtube.com/watch?v=OAUnxDguldw"            # If video, then video content link
MEDIA_DIRECTORY = "media"                                               # media directory
OPEN_AI_KEY = ""                                                        
ELEVEN_LABS_KEY = ""

if(CONTENT == "shorts"):
  MEDIA_DIRECTORY = "media/shorts"
  MEDIA_LENGTH = 6
else:
  MEDIA_DIRECTORY = "media/normal"
  MEDIA_LENGTH = 40


TOPICS = ["power","hate","friendship","life","gossip","self control","top 10 stoic words","dark times","love","motivation during hard times","emotions"]
LANGUAGES = ["en","tr","es", "fr", "pt", "it"]
STORY_FILE = "story_{language}.txt"


In [ ]:
# If you already have media files as zip then run this script to extract them
%%capture
!cp drive/MyDrive/media.zip media.zip
!unzip media.zip

In [3]:
# Imagemagick is required for some python packages
%%capture
!apt install imagemagick
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [1]:
# Install required packages
%%capture
!pip3 install git+https://github.com/linto-ai/whisper-timestamped
!python3 -m pip install git+https://github.com/pytube/pytube
!pip3 install openai
!pip install ffmpeg-python
!pip install mutagen
!pip install opencv-python

In [ ]:
import random
import whisper_timestamped as whisper
from pytube import YouTube
import json
from openai import OpenAI
from mutagen.mp3 import MP3
import requests
import os
from PIL import Image
import time
from moviepy.editor import *
import moviepy.editor as mp
import ffmpeg
import cv2
from moviepy.editor import TextClip, CompositeVideoClip, ColorClip
from moviepy.editor import concatenate_videoclips
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import multiprocessing


random.seed(time.time())

def gpt_generate():
  client = OpenAI(api_key=OPEN_AI_KEY)
  topic = random.choice(TOPICS)
  response = client.chat.completions.create(
  model="gpt-4",
  messages=[
      {"role": "system", "content": "You are a intelligent assistant."},
      {"role": "user", "content": f"Please write me an interesting and engagind stoic story about {topic}. Story must start explaining importance of {topic} and then start story itself and finish with conclusion. Whole story should be around 1500 words long and must contain stoicism, story took place in ancient times Rome or Greece."}
  ]
  )
  result = response.choices[0].message.content
  return result

# If content lenght is not sufficient then run this function or if the source is from video not gpt.
def gpt_correct_youtube(content):
  client = OpenAI(api_key=OPEN_AI_KEY)
  _model = "gpt-4" 
  response = client.chat.completions.create(
  model= _model,
  messages=[
      {"role": "system", "content": "You are a intelligent assistant."} ,
      {"role": "user", "content": f"Create 1500 words long content from this:  {content}"},
  ])
  result = response.choices[0].message.content
  return result

# Generate short content script 
def gpt_generate_short():
  client = OpenAI(api_key=OPEN_AI_KEY)
  _model = "gpt-3.5-turbo"
  response = client.chat.completions.create(
  model= _model,
  messages=[
    {"role": "user", "content": f"make a stoic content related to {random.choice(TOPICS)}. Should be around 110 words long, use simple terms."},
    ])
  result = response.choices[0].message.content
  return result




Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [36]:
# If source is video then extract subtitles
def download_from_youtube():
  yt = YouTube(YOUTUBE_LINK)
  yt.bypass_age_gate()

  output = ""
  result = yt.captions["a.en"].json_captions
  for y in result["events"]:
    if(not "segs" in y):
      continue
    for x in y["segs"]:
      output = output + x["utf8"]
  return output

In [ ]:
# ------------------- Content generation ---------------------------

final_result = ""

if(CONTENT == "shorts" and YOUTUBE_LINK == ""):
  final_result = gpt_generate_short()
elif (YOUTUBE_LINK != ""):
  result = download_from_youtube()
  result = result.replace("\n" , " ")
  final_result = gpt_correct_youtube(result)
else:
  final_result = gpt_generate()

with open(STORY_FILE.format(language="en"),"w",encoding= "utf-8") as file:
  file.write(final_result)

print(final_result)

In [ ]:
def text_to_speech(text,file_path):
  CHUNK_SIZE = 1024
  url = "https://api.elevenlabs.io/v1/text-to-speech/iUPqiv5ngkPsV4qTeekP"
  headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": ELEVEN_LABS_KEY
  }

  data = {
  "text": text,
  "model_id": "eleven_multilingual_v2",
  "voice_settings": {
    "stability": 0.89,
    "similarity_boost": 0.91,
    "style": 0.17,
   }
  }
  response = requests.post(url, json=data, headers=headers)
  print(response.status_code,response.text[:100])
  with open(file_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

In [57]:
# ----------------------- Text to Speech ----------------------------

def random_char_generator():
  random.seed(time.time())
  return "".join([random.choice("abcdefrtyquipzor") for _ in range(6)])

content = ""
lang_directory = {}

for lang in LANGUAGES:
  if(not os.path.exists(STORY_FILE.format(language=lang))):
    continue
  with open(STORY_FILE.format(language=lang),"r",encoding= "utf-8") as file:
    content = file.read()
  pin = random_char_generator()
  directory = f"story_{pin}_{lang}"
  os.mkdir(directory)
  lang_directory[lang] = directory

  index = 0
  counter = 0
  while(index < len(content.split("."))):
    frame = ""
    counter += 1
    soundtrack_path = os.path.join(directory,str(counter)+".mp3")
    while(len(frame)<2000 and index < len(content.split("."))):
      frame = frame + content.split(".")[index] + "."
      index += 1
    text_to_speech(frame,soundtrack_path)


200 ����  e��L����b���sp�S	��[��%-�^LZ1X�0(@@A�&L�2dɓ&@� �,�2d��Ј�� �2dɓ'v@����ww�h��
��:�cNI����8  �wwg�&MB!�ɓ<�4Ј�L�2dɓNЈ B" �����DDDx�<�2i��DB
200 ����  (�,= j�?�Ԏy >�P��gA�.��@h(y���Y∈�������勋�b"""#�����"""!�  � �����    �����    ����  


In [58]:
# ------------------------- Combine all audio files -----------------------

def concat_mp3s(mp3s, output):
    input_args = []
    for mp3 in mp3s:
        input_args.append(ffmpeg.input(mp3))
    ffmpeg.concat(*input_args, v=0, a=1).output(output).run()

for lang,directory in lang_directory.items():
  mp3s = map(lambda x: os.path.join(directory, x) , sorted(os.listdir(directory), key = lambda x: int(x.split(".")[0])))
  mp3s = list(mp3s)
  final_path = os.path.join(directory, "final.mp3")
  if(len(mp3s) == 1):
    os.rename(mp3s[0],final_path)
  else:
    concat_mp3s(mp3s, final_path)


In [7]:
# ------------------------ Transcribe -----------------------

def run_whisper(filename,language):
  MODEL = whisper.load_model("medium")
  audio = whisper.load_audio(filename)
  result = whisper.transcribe(MODEL, audio, language = language, beam_size=5, best_of=5, temperature=(0.0, 0.2, 0.4, 0.6, 0.8, 1.0))
  with open(f"subtitle_{language}.json","w", encoding='utf-8') as _file:
    json.dump(result, _file, ensure_ascii=False, indent=4)



def get_transcription():
  for language in LANGUAGES:
    if(not os.path.exists(f"subtitle_{language}.json")):
      continue
    output = []
    json_data = None
    with open(f"subtitle_{language}.json","r", encoding='utf-8') as _file:
      json_data = json.load(_file)

    for segment in json_data["segments"]:
      for word in segment["words"]:
        output.append({"start":word["start"],"end":word["end"],"word":word["text"]})
    yield (language,output)



def split_text_into_lines(data):

    MaxChars = 30
    #maxduration in seconds
    MaxDuration = 2.5
    #Split if nothing is spoken (gap) for these many seconds
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0


    for idx,word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"] for item in line)


        # Check if adding a new word exceeds the maximum character count or duration
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars
        if idx>0:
          gap = word_data['start'] - data[idx-1]['end']
          # print (word,start,end,gap)
          maxgap_exceeded = gap > MaxGap
        else:
          maxgap_exceeded = False


        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0


    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

#lang_directory = {"en":"en"}

for lang,directory in lang_directory.items():
  run_whisper(os.path.join(directory,"final.mp3"),language=lang)


linelevel_subtitles = {}
for language,data in get_transcription():
  if(data is None):
    raise Exception("Files not found !")
  linelevel_subtitles[language] = split_text_into_lines(data)



100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 166MiB/s]
100%|██████████| 35312/35312 [01:17<00:00, 455.30frames/s]


In [8]:

def create_caption(textJSON, framesize,font = "Helvetica",color='white', highlight_color='yellow',stroke_color='black',stroke_width=0.4):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end']-textJSON['start']

    word_clips = []
    xy_textclips_positions =[]

    x_pos = 0
    y_pos = 0
    line_width = 0  # Total width of words in the current line
    frame_width = framesize[0]
    frame_height = framesize[1]

    x_buffer = frame_width*1/12

    max_line_width = frame_width - 2 * (x_buffer)

    fontsize = int(frame_height * 0.055) #7.5 percent of video height

    space_width = ""
    space_height = ""

    for index,wordJSON in enumerate(textJSON['textcontents']):
      duration = wordJSON['end']-wordJSON['start']
      word_clip = TextClip(wordJSON['word'], font = font,fontsize=fontsize, color=color,stroke_color=stroke_color,stroke_width=stroke_width).set_start(textJSON['start']).set_duration(full_duration)
      word_clip_space = TextClip(" ", font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_width, word_height = word_clip.size
      space_width,space_height = word_clip_space.size
      if line_width + word_width+ space_width <= max_line_width:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width, y_pos))

            x_pos = x_pos + word_width+ space_width
            line_width = line_width+ word_width + space_width
      else:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos+ word_height+10
            line_width = word_width + space_width

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" :  word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width , y_pos))
            x_pos = word_width + space_width


      word_clips.append(word_clip)
      word_clips.append(word_clip_space)


    for highlight_word in xy_textclips_positions:

      word_clip_highlight = TextClip(highlight_word['word'], font = font,fontsize=fontsize, color=highlight_color,stroke_color=stroke_color,stroke_width=stroke_width).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
      word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
      word_clips.append(word_clip_highlight)

    return word_clips,xy_textclips_positions



def add_captions_to_video(video_path,captions,output_path):
  input_video = VideoFileClip(video_path)
  frame_size = input_video.size
  all_linelevel_splits=[]

  for line in captions:
    out_clips,positions = create_caption(line,frame_size)

    max_width = 0
    max_height = 0

    for position in positions:
      x_pos, y_pos = position['x_pos'],position['y_pos']
      width, height = position['width'],position['height']

      max_width = max(max_width, x_pos + width)
      max_height = max(max_height, y_pos + height)

    color_clip = ColorClip(size=(int(max_width*1.1), int(max_height*1.1)),
                       color=(64, 64, 64))
    color_clip = color_clip.set_opacity(.1)
    color_clip = color_clip.set_start(line['start']).set_duration(line['end']-line['start'])

    #centered_clips = [each.set_position('center') for each in out_clips]

    clip_to_overlay = CompositeVideoClip([color_clip]+out_clips)
    #clip_to_overlay = clip_to_overlay.set_position("bottom")

    all_linelevel_splits.append(clip_to_overlay.set_position("center"))

  input_video_duration = input_video.duration


  final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

  # Set the audio of the final video to be the same as the input video
  final_video = final_video.set_audio(input_video.audio)

  # Save the final clip as a video file with the audio included
  final_video.write_videofile(output_path, threads = 8 fps=24, codec="libx264", audio_codec="aac")




In [9]:
# ----------------------- Pre-Process Images/Videos -----------------
# You can convert files to grayscale and change its size as well.



def is_imagefile(name):
  if( name.split(".")[-1] in ["png","jpeg","jpg"]):
    return True
  return False

def is_videofile(name):
  if( name.split(".")[-1] in ["mp4","avi","wmv"]):
    return True
  return False

def is_audiofile(name):
  if( name.split(".")[-1] in ["mp3"]):
    return True
  return False

def get_video_info(file_path):
  clip = VideoFileClip(file_path)
  duration       = clip.duration
  fps            = clip.fps
  width, height  = clip.size
  return duration, fps, (width, height)

def add_mark(name):
  extension = ".png" if(is_imagefile(name)) else "."+name.split(".")[-1]
  return "".join(name.split(".")[:-1])+"_edited_by_creator_"+extension


def pre_process_images():
  media_files = os.listdir(MEDIA_DIRECTORY)
  image_files = filter(lambda x: is_imagefile(x) and not "_edited_by_creator_" in x, media_files)
  image_files = list(map(lambda x: os.path.join(MEDIA_DIRECTORY,x), image_files))

  for image in image_files:
    img = Image.open(image)
    rgbimg = Image.new("RGBA", img.size)
    rgbimg.paste(img)
    rgbimg.save(add_mark(image))


def convert_to_grayscale(input_file, output_file):
    cap = cv2.VideoCapture(input_file)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height), isColor=False)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        out.write(gray_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()


def pre_process_videos():
  media_files = os.listdir(MEDIA_DIRECTORY)
  video_files = filter(lambda x: is_videofile(x) and not "_edited_by_creator_" in x, media_files)
  video_files = list(map(lambda x: os.path.join(MEDIA_DIRECTORY,x), video_files))

  for video in video_files:
    convert_to_grayscale(video,add_mark(video))

pre_process_images()
pre_process_videos()

In [17]:
# ------------------------ Create Video --------------------

def create_video(speech_mp3_path,subtitles,language):
  random.seed(time.time())
  STYLE = "storm" #random.choice(["storm","stone"])

  media_with_extension = STYLE+"_shorts" if(CONTENT == "shorts") else STYLE
  media_files = os.listdir(os.path.join(MEDIA_DIRECTORY,media_with_extension))
  image_files = filter(lambda x: is_imagefile(x) , media_files)
  video_files = filter(lambda x: is_videofile(x) , media_files)
  audio_files = filter(lambda x: is_audiofile(x), os.listdir(os.path.join("media","soundtracks")))

  image_files = list(map(lambda x: os.path.join(MEDIA_DIRECTORY,media_with_extension,x), image_files))
  video_files = list(map(lambda x: os.path.join(MEDIA_DIRECTORY,media_with_extension,x), video_files))
  audio_files = list(map(lambda x: os.path.join(os.path.join("media","soundtracks"),x), audio_files))

  random.shuffle(image_files)
  random.shuffle(video_files)
  random.shuffle(audio_files)

  if(len(image_files)<MEDIA_LENGTH):
    image_files = image_files + image_files[:MEDIA_LENGTH-len(image_files)]

  ### ADD SLIGHT FADE OUT EFFECT TO AUDIO NEAR END AND NEAR BEGINNING
  soundtrack = random.choice(audio_files)
  duration = MP3(speech_mp3_path).info.length

  clips = []

  if(IMAGE_VIDEO == "image"):
    duration_each_image = duration//MEDIA_LENGTH
    clips = [ImageClip(m).set_duration(duration_each_image).crossfadein(7.0)
         for m in image_files[:MEDIA_LENGTH]]
  else:
    counter = 0
    for video in video_files[:MEDIA_LENGTH]:
      vid_duration = get_video_info(video)[0]
      counter += vid_duration
      if(counter>=duration):
        diff = abs(duration-counter)
        diff = vid_duration - diff
        clips.append(VideoFileClip(video).set_duration(diff))
        break
      clips.append(VideoFileClip(video))

  concat_clip = mp.concatenate_videoclips(clips, method="compose")

  audio_clip = AudioFileClip(speech_mp3_path)
  soundtrack_clip = AudioFileClip(soundtrack).set_duration(duration+10)

  new_audioclip = CompositeAudioClip([audio_clip, soundtrack_clip])
  concat_clip.audio = new_audioclip

  concat_clip.write_videofile(f"test_without_captions_{language}.mp4", fps=24)
  #video = VideoFileClip(f"test_without_captions{language}.mp4")
  add_captions_to_video(f"test_without_captions_{language}.mp4",subtitles,f"output_{language}.mp4")
  return f"Completed successfully ! >> output_{language}.mp4"

for lang,directory in lang_directory.items():
  create_video(os.path.join(directory,"final.mp3"),linelevel_subtitles[lang],lang)




threads = []
with ThreadPoolExecutor(max_workers = 3) as executor:
  for lang,directory in lang_directory.items():
    directory = os.path.join(directory,"final.mp3")
    threads.append(executor.submit(create_video, directory,linelevel_subtitles[lang],lang))

for thread in threads:
  print(thread.result())


Moviepy - Building video test_without_captions_en.mp4.
MoviePy - Writing audio in test_without_captions_enTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test_without_captions_en.mp4



Moviepy - Done !
Moviepy - video ready test_without_captions_en.mp4
Moviepy - Building video output_en.mp4.
MoviePy - Writing audio in output_enTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_en.mp4



Streaming output truncated to the last 5000 lines.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  81%|████████  | 7053/8717 [24:39<03:09,  8.77it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file test_without_captions_en.mp4, 3096576 bytes wanted but 0 bytes read,at frame 7052/8717, at time 293.83/363.18 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  81%|████████  | 7054/8717 [24:39<03:08,  8.83it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file test_without_captions_en.mp4, 3096576 bytes wanted but 0 bytes read,at frame 7053/8717, at time 293.88/363.18 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  81%|████████  | 7055/8717 [24:39<03:05,  8.95it/s, now=None]WARNING:py.warnings:/us

Moviepy - Done !
Moviepy - video ready output_en.mp4
Completed successfully ! >> output_en.mp4
